Item 34 Avoid Injecting Data into Generators with send

Things to Remember
- The send method can be used to inject data into a generator by giving the yield expression a value that can be assigned to a variable.
- Using send with yield from expression may cause surprising behavior, such as None values appearing at unexpected times in the generator output
- Providing an input iterator to a set of composed generators is a better approach than using the send method, which should be avoided.

Background
- yield expressions provide generator functions with a simple way to produce an iterable series of output values
- however, this channel appears to be unidirectional: there is no immediately obvious way to simultaneously stream data in and out a generator as it runs 

In [ ]:
# - you're writing a program to transmit signals using 
#   a software-defined radio
# - the following function will generate an approximation
#   of  a sine wave with a given number of points

import math

def wave(amplitude, steps):
    '''
        amplitude: the strength of the wave
        steps: number of points used to generate an approximation of a sine wave
    '''
    step_size = 2 * math.pi / steps # 2 * math.pi is a whole cycle
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        yield output

In [ ]:
def transmit(output):
    if output is None:
        print(f'Output is None')
    else:
        print(f'Output: {output:>5.1f}')
def run(it):
    for output in it:
        transmit(output)
run(wave(3.0, 8)) # points are generated under the same amplitude (3.0)

understand the value of yield expression
- it's not the output generated by the yield expression.
- normally, when iterating a generator, the value of the yield expression is None.

In [ ]:
def my_generator():
    received = yield 1 # the 'received' variable holds the value of the yield expression
    print(f'received = {received}') # received is None

In [ ]:
it = iter(my_generator())
output = next(it)
print(f'output = {output}')
try:
    next(it)
except StopIteration:
    pass

Understand the send method
- upgrades yield expressions into a two-way channel
- can be used to provide streaming inputs to a generator at the same time it's yielding outputs
- the supplied parameter in the send method becomes the value of the yield expression when the generator is resumed
- when the generator first starts, a yield expression has not been encountered yet, so the only valid value for calling send initially is None

In [ ]:
it = iter(my_generator())
# initially the parameter of the send method has to be None
output = it.send(None) # use send instead of next to get the output 
print(f'output = {output}')
try:
    # -  resume the generator to get 
    #    the output again
    # -  at which point the 'hello!' will become
    #    the value of the yield expression
except StopIteration:
    it.send('hello!')
    pass

In [ ]:
# - now we are ready to modify the wave method to generate points 
#   with varying amplitudes within an iteration of the generator 

def wave_modulating(steps):
    step_size = 2 * math.pi / steps
    # - the first call to the send will stop
    #   after the yield expression
    # - the second call to the send will resume
    #   the generator and set the amplitude to 
    #   the same as the supplied paramter 
    #   in the send method 
    amplitude = yield # receive initial amplitude 
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        amplitude = yield output # receive next amplitude

In [ ]:
def run_modulating(it):
    # - the first input has to be None, since a yield
    #   expression would not have occurred yet
    # - a yield expression would consider occurred 
    #   once the generator is resumed 
    amplitudes = [
        None, 7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10]
    for amplitude in amplitudes:
        output = it.send(amplitude)
        transmit(output)
run_modulating(wave_modulating(12))

Problems with the above approach
- using yield on the right side of an assignment statement isn't intuitive
- hard to see the connection between yield and send without knowing the details of this advanced generator feature
- cause unexpected behavior in more complex scenarios (check below) 

In [ ]:
# compose multiple generators to create a complex waveform
def complex_wave():
    yield from wave(7.0, 3)
    yield from wave(2.0, 4)
    yield from wave(10.0, 5)
# - the output has 12 points
# - use this as a baseline to compare
#   to the send method approach below
run(complex_wave())

In [ ]:
def complex_wave_modulating():
    yield from wave_modulating(3) # nested generator
    yield from wave_modulating(4)
    yield from wave_modulating(5)
run_modulating(complex_wave_modulating())

Problems with the above approach
- There are short of two points - only 10 as opposed to 12.
- There are many None values in the output
- This is because each nested generator starts with a bare yield expression, one without an output value, in order to receive the initial amplitude from a generator send method call
- As a result, the parent generator outputs a None value when it transitions between child generators
- Generally speaking avoid using send and the yield from expressions together   

In [ ]:
# Solution
# - pass an iterator into the wave function 
# - the iterator should return an input amplitude 
#   each time the next built-in is called on it     

In [ ]:
def wave_cascading(amplitude_it, steps):
    step_size = 2 * math.pi / steps
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        amplitude = next(amplitude_it)
        output = amplitude * fraction
        yield output

In [ ]:
def complex_wave_cascading(amplitude_it):
    yield from wave_cascading(amplitude_it, 3)
    yield from wave_cascading(amplitude_it, 4)
    yield from wave_cascading(amplitude_it, 5)

In [ ]:
def run_cascading():
    amplitudes = [7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10]
    it = complex_wave_cascading(iter(amplitudes))
    for amplitude in amplitudes:
        output = next(it)
        transmit(output)

run_cascading()

- the iterator can come from anywhere and could be completely dynamic
- be aware that the code assumes the input generator is completely thread safe (check item 62 for more info)